## Initializing Python

In [2]:
#!/usr/bin/env python
# -*- coding: UTF-8

In [1]:
# IMPORTING KEY PACKAGES
import csv # for reading in CSVs and turning them into dictionaries
import re # for regular expressions
import os # for navigating file trees
import nltk # for natural language processing tools
import pandas # for working with dataframes
import numpy as np # for working with numbers

In [2]:
# FOR CLEANING, TOKENIZING, AND STEMMING THE TEXT
# from nltk import word_tokenize, sent_tokenize # widely used text tokenizer
# from nltk.stem.porter import PorterStemmer # an approximate method of stemming words (it just cuts off the ends)
# from nltk.corpus import stopwords # for one method of eliminating stop words, to clean the text
# stopenglish = list(stopwords.words("english")) # assign the string of english stopwords to a variable and turn it into a list
import string # for one method of eliminating punctuation
punctuations = list(string.punctuation) # assign the string of common punctuation symbols to a variable and turn it into a list
from scipy.stats.stats import pearsonr

In [3]:
# FOR VISUALIZATIONS
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Visualization parameters
% pylab inline 
% matplotlib inline
matplotlib.style.use('ggplot')

Populating the interactive namespace from numpy and matplotlib


## Reading in preliminary data

In [4]:
# Read the newly merged data in as a pandas dataframe
df = pandas.read_csv("withPVI_new.csv", encoding = 'Latin-1')
# df = df.dropna(subset=["WEBTEXT"]) # drop any schools with no webtext that might have snuck in (none currently)

/Users/shiji/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,2,3,4,6,7,9,11,12,13,14,15,16,20,21,22,23,24,28,29,31,32,33,34,35,37,39,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,67,269,270,271,272,352,353,354,355,356,357,358,359,361,362,363,365,366,368,369,371,372,374,375,377,378,380,381,383,384,386,387,389,390,392,393,395,396,398,399,401,402,403,404,406,408,409,411,413,414,416,418,419,421,422,423,425) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Using PVI Scores
Here we'll use Partisan Voting Index scores instead of voting records to assess political preference
PVI scores are based on congressional district, but the data we have does not say which congressional district a school belongs to.  So, we will use zip codes to map schools to their congressional districts.

In [6]:
def strip_leading_zero(s):
    if s[0] == "0" and len(s) > 1:
        return s[1:]
    return s

# Get mapping from zip code to congressional district
zip_dict = dict()
with open("zipcode_to_cd.txt") as f:
    line = f.readline().replace("\n", "")
    while line:
        line = line.split(",")
        line = [strip_leading_zero(el) for el in line]
        zip_dict[line[1]] = line[0] + "," + line[2]
        line = f.readline().replace("\n", "")

# Get mapping from congressional district to PVI score
# Note that negative scores indicate a Democratic score and positive scores indicate a Republican score
pvi_dict14 = dict()
with open("pvi_2014.txt") as f:
    line = f.readline().replace("\n", "")
    while line:
        line = line.split()
        line = [strip_leading_zero(el) for el in line]
        pvi_dict14[line[0] + "," + line[1]] = line[2]
        line = f.readline().replace("\n", "")

pvi_dict17 = dict()
with open("pvi2017_n.txt") as f:
    line = f.readline().replace("\n", "")
    while line:
        line = line.split()
        line = [strip_leading_zero(el) for el in line]
        pvi_dict17[line[0] + "," + line[1]] = line[2]
        line = f.readline().replace("\n", "")

In [7]:
# Get Berkeley's PVI score
print(zip_dict['94709'])
print(zip_dict['94709'])
#print(pvi_dict)
print(pvi_dict14[zip_dict['94704']])
print(pvi_dict17[zip_dict['94704']])

6,13
6,13
-37
-40


In [8]:
# Map schools to their PVI scores and store in a "PVI" column
pvis, not_found, = [], 0
for index, row in df.iterrows():
    if not math.isnan(row["LZIP"]):
        v = int(row["LZIP"])
    else:
        pvis.append('NA')
        continue
    if str(v) not in zip_dict:
        pvis.append('NA') # For now defaulting to 0 when zip code not found
        not_found += 1
    else:
        if zip_dict[str(v)] not in pvi_dict14:
            pvis.append('NA')
            continue
        pvi_value = pvi_dict14[zip_dict[str(v)]]
        pvis.append(int(pvi_value))
print(str(not_found) + " zip codes not found")
df["PVI2014"] = pvis
df[["SCH_NAME", "PVI2014"]][:10]

6073 zip codes not found


,SCH_NAME,PVI2014
0,Sequoyah Sch - Chalkville Campus,NA
1,Eufaula Sch - Eufaula Campus,17
2,Camps,17
3,Det Ctr,-20
4,Wallace Sch - Mt Meigs Campus,NA
5,McNeel Sch - Vacca Campus,-20
6,Alabama Youth Services,NA
7,Ala Avenue Middle Sch,28
8,Albertville High Sch,28
9,Evans Elem Sch,28


In [9]:
pvis, not_found, = [], 0
for index, row in df.iterrows():
    if not math.isnan(row["LZIP"]):
        v = int(row["LZIP"])
    else:
        pvis.append('NA')
        continue
    if str(v) not in zip_dict:
        pvis.append('NA') # For now defaulting to 0 when zip code not found
        not_found += 1
    else:
        if zip_dict[str(v)] not in pvi_dict17:
            pvis.append('NA')
            continue
        pvi_value = pvi_dict17[zip_dict[str(v)]]
        pvis.append(int(pvi_value))
print(str(not_found) + " zip codes not found")
df["PVI2017"] = pvis
df[["SCH_NAME", "PVI2014", "PVI2017"]][:50]

6073 zip codes not found


,SCH_NAME,PVI2014,PVI2017
0,Sequoyah Sch - Chalkville Campus,NA,NA
1,Eufaula Sch - Eufaula Campus,17,16
2,Camps,17,16
3,Det Ctr,-20,-20
4,Wallace Sch - Mt Meigs Campus,NA,NA
5,McNeel Sch - Vacca Campus,-20,-20
6,Alabama Youth Services,NA,NA
7,Ala Avenue Middle Sch,28,30
8,Albertville High Sch,28,30
9,Evans Elem Sch,28,30


In [28]:
print(df)

         SURVYEAR  FIPST STABR STATENAME                          SEANAME  \
0       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
1       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
2       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
3       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
4       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
5       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
6       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
7       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
8       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
9       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
10      2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
11      2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   

In [11]:
df.to_csv("withPVI_final.csv", index = False)